In [129]:
from collections import Counter
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords
import re
import requests
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

In [130]:
# Download NLTK resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [131]:
from transformers import pipeline
docs =[]
topics = ["supervised learning","unsupervised learning","Reinforcement learning"]
for topic in topics:
  # Initialize the text generation pipeline with the desired model
  generator = pipeline('text-generation', model='gpt2')
  prompt = topic
  generated_text = generator(topic, max_length=50)
  docs.append(generated_text)


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=

In [136]:
print(docs)

[[{'generated_text': 'supervised learning for high-performance computing. Based on this understanding of the theoretical and practical applications of neural networks, we propose a computer operating system that can implement both network security and deep learning to enhance the security of computerized AI systems.\n\n'}], [{'generated_text': 'unsupervised learning, a method that might provide better information on future trends in artificial intelligence.\n\nAnd, even with that potential, the project is still under an aggressive development stage, with more questions than answers from what the algorithms can learn and'}], [{'generated_text': "Reinforcement learning is a key driver of a large social network, meaning that a strong interest in learning is essential as we build an effective social network. We've been thinking about reinstating reinforcement learning while exploring the concept of neural-machine learning"}]]


In [126]:
# Apply cleaning function to each document
unique_words = []
documents_=[]
for doc in docs:
    for sub_doc in doc:
        for key, value in sub_doc.items():
            # Normalization
            cleaned_data = re.sub(r'[^a-zA-Z\s]', '', value)
            # Tokonization
            tokonizes = nltk.word_tokenize(cleaned_data.lower())
            # Remove stopwords
            stop_words = set(stopwords.words('english'))
            tokens = [word for word in tokonizes if word not in stop_words]
            # lemmatizing
            lemmatizer = WordNetLemmatizer()
            lemmatizer_words = [lemmatizer.lemmatize(word)for word in tokens]
            #unique words
            unique_words.append(list(set(lemmatizer_words)))
            documents_.append(lemmatizer_words)

print(unique_words[])

['youll', 'learning', 'here', 'ever', 'supervised', 'article', 'ive', 'algorithm', 'complicated', 'problem', 'interesting', 'wrote', 'often', 'think', 'would', 'solve', 'example', 'ask', 'also', 'one']


In [127]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
import numpy as np

corpus=[' '.join(doc) for doc in documents_]

# Compute TF for each word
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)

# Get vocabulary
vocab = vectorizer.get_feature_names_out()

#Calculate IDF for each word
transformer = TfidfTransformer()
transformer.fit(X)

# Get IDF values
idf_values = transformer.idf_

#Compute TF * IDF
tfidf_matrix = np.zeros((len(corpus), len(vocab)))
for i in range(len(corpus)):
    for j, word in enumerate(vocab):
        tfidf_matrix[i][j] = X[i, vectorizer.vocabulary_.get(word)] * idf_values[vectorizer.vocabulary_.get(word)]

# Normalize TF-IDF for each document
normalized_tfidf_matrix = np.zeros_like(tfidf_matrix)

for i in range(len(corpus)):
    tfidf_sum_squares = np.sqrt(np.sum(np.square(tfidf_matrix[i])))
    normalized_tfidf_matrix[i] = tfidf_matrix[i] / tfidf_sum_squares

print(normalized_tfidf_matrix)

[[0.20933872 0.20933872 0.20933872 0.20933872 0.         0.20933872
  0.         0.         0.20933872 0.31841498 0.         0.
  0.20933872 0.         0.20933872 0.20933872 0.         0.
  0.         0.12363882 0.         0.         0.         0.20933872
  0.15920749 0.         0.41867744 0.         0.         0.
  0.20933872 0.         0.15920749 0.         0.20933872 0.
  0.         0.         0.         0.         0.         0.20933872
  0.20933872 0.20933872]
 [0.         0.         0.         0.         0.         0.
  0.23652731 0.23652731 0.         0.17988511 0.23652731 0.
  0.         0.23652731 0.         0.         0.         0.
  0.         0.41909052 0.23652731 0.         0.         0.
  0.         0.         0.         0.17988511 0.23652731 0.47305462
  0.         0.         0.17988511 0.         0.         0.
  0.         0.23652731 0.23652731 0.23652731 0.         0.
  0.         0.        ]
 [0.         0.         0.         0.         0.17815825 0.
  0.         0.   

# **From scratch**

In [128]:
import numpy as np
import math

def count_vectorizer(documents):

    vocab = set()
    word_count = []

    for doc in documents:
        words = doc.split()
        word_count_doc = {}
        for word in words:
            if word not in word_count_doc:
                word_count_doc[word] = 1
            else:
                word_count_doc[word] += 1
            vocab.add(word)
        word_count.append(word_count_doc)

    vocab = list(vocab)
    vocab.sort()

    return word_count, vocab

def compute_idf(documents, vocab):

    idf_values = {}

    for word in vocab:
        doc_count = sum(1 for doc in documents if word in doc)
        idf_values[word] = np.log((len(documents)+1) / (1 + doc_count)) +1

    return idf_values

def compute_tfidf(word_count, idf_values, vocab):

    tfidf_matrix = np.zeros((len(word_count), len(vocab)))

    for i, word_count_doc in enumerate(word_count):
        for j, word in enumerate(vocab):
            tf = word_count_doc.get(word, 0)
            tfidf_matrix[i][j] = tf * idf_values[word]

    return tfidf_matrix

def normalize_tfidf(tfidf_matrix):

    # Normalize TF-IDF for each document
    normalized_tfidf_matrix = np.zeros_like(tfidf_matrix)

    for i in range(len(corpus)):
         tfidf_sum_squares = np.sqrt(np.sum(np.square(tfidf_matrix[i])))
         normalized_tfidf_matrix[i] = tfidf_matrix[i] / tfidf_sum_squares
    return normalized_tfidf_matrix


#compute TF for each word
word_count, vocab = count_vectorizer(corpus)

#calculate IDF for each word
idf_values = compute_idf(corpus, vocab)

# compute TF * IDF
tfidf_matrix = compute_tfidf(word_count, idf_values, vocab)

#normalize TF-IDF for each document
normalized_tfidf_matrix = normalize_tfidf(tfidf_matrix)

print(normalized_tfidf_matrix)


[[0.20933872 0.20933872 0.20933872 0.20933872 0.         0.20933872
  0.         0.         0.20933872 0.31841498 0.         0.
  0.20933872 0.         0.20933872 0.20933872 0.         0.
  0.         0.12363882 0.         0.         0.         0.20933872
  0.15920749 0.         0.41867744 0.         0.         0.
  0.20933872 0.         0.15920749 0.         0.20933872 0.
  0.         0.         0.         0.         0.         0.20933872
  0.20933872 0.20933872]
 [0.         0.         0.         0.         0.         0.
  0.23652731 0.23652731 0.         0.17988511 0.23652731 0.
  0.         0.23652731 0.         0.         0.         0.
  0.         0.41909052 0.23652731 0.         0.         0.
  0.         0.         0.         0.17988511 0.23652731 0.47305462
  0.         0.         0.17988511 0.         0.         0.
  0.         0.23652731 0.23652731 0.23652731 0.         0.
  0.         0.        ]
 [0.         0.         0.         0.         0.18002842 0.
  0.         0.   